# Notebook 3: Data Processing and Clustering with `scanpy`

**Tutor:** Anthony Christidis
**Time:** 45 minutes

---

Welcome to the computational analysis part of the workshop! Before we dive into advanced spatial statistics, we must first process our raw gene expression data to identify meaningful biological groupings. This is a fundamental step in almost any single-cell or spatial analysis.

In this notebook, we'll use `scanpy` to perform a standard clustering analysis and `matplotlib` for robust QC plotting. We will first learn the workflow in detail on a **10x Visium** dataset, and then apply the same principles to prepare our **Xenium** data for the next notebook.

**Goals:**
1.  Perform a comprehensive Quality Control (QC) workflow on Visium data.
2.  Run a standard unsupervised clustering workflow (`scanpy`).
3.  Visualize the final spot clusters on the tissue, confirming that our analysis reveals underlying biology.
4.  Apply a streamlined workflow to process our Xenium data.

### Setup

First, we'll import our libraries and load the Visium Glioblastoma dataset.

In [1]:
import spatialdata as sd
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D

import warnings
warnings.filterwarnings("ignore")

sdata_visium = sd.read_zarr("../data/visium_glioblastoma_subset.zarr")
adata_visium = sdata_visium.tables["table"]

/Users/tim.treis/Documents/GitHub/Spatial-Workshop/.pixi/envs/default/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/Users/tim.treis/Documents/GitHub/Spatial-Workshop/.pixi/envs/default/lib/python3.11/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Part 1: Visium Analysis - Spatial Quality Control

For spot-based data like Visium, visualizing QC metrics spatially is a critical first step. It can reveal technical issues like tissue detachment or slide artifacts.

First, we calculate standard QC metrics, such as the number of genes detected per spot (`n_genes_by_counts`) and the total number of transcripts per spot (`total_counts`).

In [ ]:
sc.pp.calculate_qc_metrics(adata_visium, percent_top=(20, 50), inplace=True)

Now, we will create spatial scatter plots to visualize these QC metrics. This allows us to see if low-quality spots are concentrated in a specific area, which might indicate a problem with the tissue section.

In [ ]:
# Get the spatial coordinates directly from the AnnData object
coords = adata_visium.obsm['spatial']
x_coords = coords[:, 0]
y_coords = coords[:, 1]

# Create the two-panel plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Total counts
scatter1 = axes[0].scatter(x_coords, y_coords, c=adata_visium.obs['total_counts'], 
                          cmap='viridis', s=25, alpha=0.8)
axes[0].set_title('Total Counts per Spot')
axes[0].set_aspect('equal')
plt.colorbar(scatter1, ax=axes[0])

# Plot 2: Number of genes
scatter2 = axes[1].scatter(x_coords, y_coords, c=adata_visium.obs['n_genes_by_counts'], 
                          cmap='viridis', s=25, alpha=0.8)
axes[1].set_title('Unique Genes per Spot')
axes[1].set_aspect('equal')
plt.colorbar(scatter2, ax=axes[1])

plt.tight_layout()
plt.show()

These plots are essential. We can see clear spatial patterns in both the total counts and the number of genes, which likely correspond to different biological regions within the glioblastoma tissue.

### Part 2: Visium Analysis - The `scanpy` Clustering Workflow

Based on our QC, let's filter out the lowest-quality spots and then run the standard `scanpy` workflow to find transcriptionally distinct groups of spots. Each step is broken down into its own cell for clarity.

#### Step 2.1: Filtering
We remove spots with very few counts and genes that are detected in very few spots. This reduces noise in our data.

In [ ]:
print(f"Spots before filtering: {adata_visium.n_obs}")
sc.pp.filter_cells(adata_visium, min_counts=500)
sc.pp.filter_genes(adata_visium, min_cells=10)
print(f"Spots after filtering: {adata_visium.n_obs}")

#### Step 2.2: Normalization and Log-Transformation
This step corrects for differences in sequencing depth between spots, ensuring that we are comparing their relative gene expression profiles.

In [ ]:
sc.pp.normalize_total(adata_visium, inplace=True)
sc.pp.log1p(adata_visium)

#### Step 2.3: Finding Highly Variable Genes (HVGs)
We don't need to use all ~18,000 genes for clustering. We can identify the genes that show the most biological variability across the tissue and focus our analysis on them. This reduces computational time and often improves results.

In [ ]:
sc.pp.highly_variable_genes(adata_visium)

#### Step 2.4: Principal Component Analysis (PCA)
PCA is a dimensionality reduction technique. We use it to summarize the main axes of variation in our highly variable genes into a smaller number of principal components (PCs).

In [ ]:
sc.pp.pca(adata_visium, use_highly_variable=True)

#### Step 2.5: Neighborhood Graph and Leiden Clustering
Next, we build a graph where each spot is a node, and nodes are connected if they are similar to each other in the PCA space. The Leiden algorithm then walks through this graph to find communities of spots that are highly interconnected. These communities are our cell type clusters.

In [ ]:
sc.pp.neighbors(adata_visium)
sc.tl.leiden(adata_visium, key_added="clusters")

#### Step 2.6: UMAP for Visualization
Finally, we compute a UMAP (Uniform Manifold Approximation and Projection). This takes our high-dimensional neighborhood graph and creates a 2D representation of it, which is useful for visualizing the relationships between our clusters.

In [ ]:
sc.tl.umap(adata_visium)

### Part 3: Visualizing the Visium Results
Let's visualize the clusters we found, both in the abstract UMAP space and back on the tissue.

In [ ]:
sc.pl.umap(adata_visium, color="clusters", title="Spot Clusters (UMAP)")

In [ ]:
# To plot spatially, we'll use our robust matplotlib method again.

# Get coordinates and cluster info
coords = adata_visium.obsm['spatial']
x_coords = coords[:, 0]
y_coords = coords[:, 1]
cluster_codes = adata_visium.obs['clusters'].astype('category').cat.codes

# Create the plot
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
scatter = ax.scatter(x_coords, y_coords, c=cluster_codes, 
                    cmap='tab10', s=25, alpha=0.8)
ax.set_title('Spot Clusters (Spatial View)')
ax.set_aspect('equal')

# Add a legend
unique_clusters = adata_visium.obs['clusters'].unique()
colors = plt.cm.tab10(np.linspace(0, 1, len(unique_clusters)))
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=colors[i], 
                         markersize=8, label=f'Cluster {cluster}') 
                  for i, cluster in enumerate(unique_clusters)]
ax.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

### Optional: Interactive QC Visualization with `napari`

While static plots are great for a quick overview, we can use `napari` for a more dynamic exploration of our QC metrics. This allows us to zoom in on specific areas and see how metrics vary across the tissue.

*(Instructor Note: I will run this live. For those using the workshop's Docker container, this requires the graphics server setup detailed in the README. You can simply watch my screen for this brief demonstration.)*

In [ ]:
import napari_spatialdata as nsd

# We launch napari with our sdata_visium object.
# The QC metrics we calculated are already in its associated table.
# viewer = nsd.Interactive(sdata_visium)

# In the Napari window that opens, you can now:
# 1. Add the hires_image and the shapes layer.
# 2. In the annotation panel on the right, under "Observation", select 'total_counts'
#    or 'n_genes_by_counts' to color the spots by these QC metrics.

### Part 4: Preparing the Xenium Data for the Next Notebook

Now that we've mastered the workflow, we will apply the same steps to our Xenium data. We will break it down into cells and include the QC histograms and the final UMAP plot, as these are known to work.

In [ ]:
print("--- Processing Xenium Data ---")

# 1. Load Data
sdata_xenium = sd.read_zarr("../data/xenium_lung_cancer_subset.zarr")
adata_xenium = sdata_xenium.tables["table"].copy()

In [ ]:
import seaborn as sns

# 2. Calculate and Visualize QC Metrics
sc.pp.calculate_qc_metrics(adata_xenium, percent_top=(20, 50), inplace=True)

fig, axs = plt.subplots(1, 2, figsize=(10, 4))
sns.histplot(adata_xenium.obs["total_counts"], kde=False, bins=100, ax=axs[0])
axs[0].set_title("Total Counts per Cell")
sns.histplot(adata_xenium.obs["n_genes_by_counts"], kde=False, bins=100, ax=axs[1])
axs[1].set_title("Unique Genes per Cell")
plt.tight_layout()
plt.show()

In [ ]:
# 3. Filter the data
print(f"Cells before filtering: {adata_xenium.n_obs}")
sc.pp.filter_cells(adata_xenium, min_counts=50)
sc.pp.filter_genes(adata_xenium, min_cells=10)
print(f"Cells after filtering: {adata_xenium.n_obs}")

In [ ]:
# 4. Run the rest of the workflow
sc.pp.normalize_total(adata_xenium)
sc.pp.log1p(adata_xenium)
sc.pp.highly_variable_genes(adata_xenium, n_top_genes=2000, flavor='seurat')
sc.pp.pca(adata_xenium, use_highly_variable=True)
sc.pp.neighbors(adata_xenium)
sc.tl.leiden(adata_xenium, key_added="clusters")
sc.tl.umap(adata_xenium)

In [ ]:
# 5. Visualize the Xenium UMAP
print(f"Xenium data processed. Found {len(adata_xenium.obs['clusters'].unique())} clusters.")
sc.pl.umap(adata_xenium, color="clusters", title="Xenium Cell Clusters (UMAP)")

In [ ]:
# 6. Save the processed AnnData object
import os
os.makedirs("../data/processed", exist_ok=True)

print("Saving processed Xenium AnnData object...")
adata_xenium.write("../data/processed/adata_xenium_processed.h5ad")
print("Done. We are ready for Notebook 4.")